In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import math
import random
import powerlaw
import colorcet as cc
import holoviews as hv
hv.notebook_extension("matplotlib")

In [ ]:
def aging_barabasi_albert_graph(N,alpha,m=2,m0=2):
    """
    Implements the modified barabasi-albert model given
    in the paper titled "Self-organized Criticality in an 
    Integrate-and-Fire Neuron Model Based on Modified Aging Networks".
    
    Each time step a random node is chosen based on its age to generate
    a new node linked to it, which also links to its (m-1) nearest neighbors
    with a probability given by the standard barabasi-albert model.
    
    N: int
        Network size
    alpha: float
        Age factor which influences probabilities based on age
    m: int
        Number of links for each new node
    m0: int
        Initial size of the network
    """
    
    # Initialize fully connected graph with m0 nodes
    graph = nx.complete_graph(m0)
    
    age = np.ones(N)
    for n in range(m0,N):
        # Get degrees for all current nodes in array form
        degrees = np.array(list(graph.degree(np.arange(n)).values()))
        
        # Calculate pmf for a node to generate new node
        i_probs = degrees / age[:n]**(alpha)
        i_probs /= np.sum(i_probs)
        
        # Choose a random node i
        i = np.random.choice(np.arange(n),p=i_probs)
        
        # neighbors j of i
        neighbors = np.array(graph.neighbors(i))
        
        # Calculate pmf for linked nearest neighbors of i
        j_probs = degrees[neighbors] / np.sum(degrees[neighbors])
        
        # Choose a random node i
        j = list(np.random.choice(neighbors,size=(m-1),replace=False,p=j_probs))
        
        # Add node to network with chosen edges
        graph.add_edges_from(zip([n]*m,[i] + j))
        
        # Advance the time and apply to age of nodes
        age[:n] += 1
    
    return graph

In [ ]:
def del_rand(adj_mat, neg_nod, per, mode):
## Function that deletes percentage of nodes or links randomly
## adj_mat: adjacency matrix of network
## neg_nod: matrix containing the negative nodes
## per: percentage of the nodes or links that will be deleted
## mode: determines whether nodes or links will be deleted
## Returns: the new adjacency matrix and the negative nodes of the network

    # Copy the adjacency and the negative nodes matrices
    am = adj_mat.copy()
    nn = neg_nod.copy()
    
    if mode == 'node':
        
        # Determine exact number of nodes
        nodes = math.ceil(adj_mat.shape[0]*per/100)
        
        for i in range(0,nodes-1):
        
            # Choose one node randomly
            n = random.randint(0, adj_mat.shape[0]-1)
            
            # Delete the corresponding row and column of the matrix            
            np.delete(am, n, 0)
            np.delete(am, n, 1)
        
            # Delete the corresponding node from the negative node matrix
            nn = [nn != n]
    
    else:

        # Determine exact number of links        
        links = math.ceil(adj_mat.shape[0]*per/100)
        
        for i in range(0,links-1):
            
            # Choose one link randomly
            l1 = random.randint(0, adj_mat.shape[0]-1)
            l2 = random.randint(0, adj_mat.shape[0]-1)
            
            # Set the corresponding cell of the matrix to zero           
            am[l1,l2] = 0
        
    
    return am, nn

In [ ]:
def izhikevich(adj_mat, neg_nodes, time=10,
               a_pars=(.02, .02), b_pars=(.2, .25),
               c_pars=(-65, 15), d_pars=(8, -6, 2),
               randomness=(0.08, 0.05)):
        #set part of neurons to inhibitory
        nnodes = adj_mat.shape[0]
        nodes = range(nnodes)
        neg_num = len(neg_nodes)

        #random distributions
        rall = np.random.rand(nnodes,1)
        re = np.random.rand(nnodes-neg_num,1)
        ri = np.random.rand(neg_num,1)
        
        #set up parameters
        a = a_pars[0]*np.ones((nnodes,1))
        a[neg_nodes] = a_pars[1]+randomness[0]*ri
        
        b = b_pars[0]*np.ones((nnodes,1))
        b[neg_nodes] = b_pars[1]-randomness[1]*ri
        
        c =  c_pars[0]+c_pars[1]*rall**2
        c[neg_nodes] = c_pars[0]*np.ones((neg_num,1))
        
        d = d_pars[0]+d_pars[1]*rall**2
        d[neg_nodes] = d_pars[2]*np.ones((neg_num,1))
        
        
        v = c_pars[0]*np.ones((nnodes,1)) # Initial values of v
        u = b*v                  
        all_act = np.zeros((nnodes, time))                    # spike timing
        
        #simulation
        for t in range(time):
            #random (sensory) input
            I=8*np.random.rand(nnodes,1)
            I[neg_nodes] = np.random.rand(neg_num,1) # input
            
            fired = np.where(v>=30)[0]
            
            all_act[fired,t] = 1
            
            v[fired]=c[fired]
            u[fired]=u[fired]+d[fired]
            
            I = I + np.sum(adj_mat[:,fired],1)
            
            v=v+0.5*(0.04*np.multiply(v,v)+5*v+140-u+I)  #step 0.5 ms for numerical stability
            v=v+0.5*(0.04*np.multiply(v,v)+5*v+140-u+I)   
            u=u+np.multiply(a,np.multiply(b,v)-u)             

        return all_act, v, u, I

def create_network(size, neighbours, frac_edge=0.1, frac_neg=0.2,
                   net_type='random'):
    if net_type == 'random':
        graph = nx.gnm_random_graph(size, neighbours)
    elif net_type == 'ws':
        graph = nx.watts_strogatz_graph(size, neighbours, frac_edge)
    elif net_type == 'barabasi':
        graph = nx.barabasi_albert_graph(size, neighbours)
    elif net_type == 'aging':
        graph = aging_barabasi_albert_graph(size,alpha=1)
    elif net_type == 'full':
        graph = nx.complete_graph(size)
    
    adj_mat = nx.to_numpy_matrix(graph)

    nodes = range(size)
    neg_nodes = np.random.choice(nodes, int(np.floor(frac_neg*size)))
    for n in neg_nodes:
        adj_mat[n,:][np.nonzero(adj_mat[n,:])] = -1
    
    return adj_mat, neg_nodes

def get_avalanches(data):
    aval_times = []
    aval = 0
    aval_sizes = []
    proj_act = np.sum(data, 0)
    for t,i in enumerate(proj_act):
        if i > 0 and aval == 0:
            aval = 1
            start_t = t
            aval_size = int(i)
        if i > 0 and aval == 1:
            aval_size += int(i)
        
        if i == 0 and aval == 1:
            aval_times.append(int(t-start_t))
            aval_sizes.append(int(aval_size))
            aval = 0
    return aval_times, aval_sizes

In [ ]:
#Plot probability of avalance size and calculate power law fit
fig = plt.figure()

colors = ['r', 'b', 'g']
for c,net_type in enumerate(['full', 'ws', 'barabasi']):
    adj_mat, neg_nodes = create_network(size=50, neighbours=5,
                                        frac_neg=0.5, net_type=net_type)
    all_act, v, u, I = izhikevich(adj_mat, neg_nodes, time=10000)

    aval_times, aval_sizes = get_avalanches(all_act)
    y = np.bincount(np.array(aval_sizes))
    ii = np.nonzero(y)[0]
    avalprobs = y/len(aval_sizes)
    plt.semilogy(ii, avalprobs[ii], colors[c] + 'o', label=net_type)
    fitted_pl = powerlaw.Fit(avalprobs[ii])
    print(fitted_pl.alpha)

plt.legend()
plt.show()

## Fully connected network

In [ ]:
# Create a fully connected network of 50 nodes and start simulation
adj_mat, neg_nodes = create_network(size=50, neighbours=5, frac_neg=0.5, net_type='full')
all_act, v, u, I = izhikevich(adj_mat, neg_nodes, time=100000)

In [ ]:
# Delete 2 percentages of nodes from network
am20, nn20 = del_rand(adj_mat, neg_nodes, 20, 'node')
am40, nn40 = del_rand(adj_mat, neg_nodes, 40, 'node')

In [ ]:
# Run the simulation for the new network
aa20, _ , _, _ = izhikevich(am20, nn20, time=100000)
aa40, _ , _, _ = izhikevich(am40, nn40, time=100000)

In [ ]:
#Plot probability of avalance size and calculate power law fit
fig = plt.figure()

aval_times, aval_sizes = get_avalanches(all_act)
y = np.bincount(np.array(aval_sizes))
ii = np.nonzero(y)[0]
avalprobs = y/len(aval_sizes)

aval_times20, aval_sizes20 = get_avalanches(aa20)
y20 = np.bincount(np.array(aval_sizes20))
ii20 = np.nonzero(y20)[0]
avalprobs20 = y20/len(aval_sizes20)

aval_times40, aval_sizes40 = get_avalanches(aa40)
y40 = np.bincount(np.array(aval_sizes40))
ii40 = np.nonzero(y40)[0]
avalprobs40 = y40/len(aval_sizes40)

plt.loglog(ii, avalprobs[ii], '*', ii20, avalprobs20[ii20], '+', ii40, avalprobs40[ii40], 'o')

fitted_pl = powerlaw.Fit(avalprobs[ii])
print(fitted_pl.alpha)
fitted_pl20 = powerlaw.Fit(avalprobs20[ii20])
print(fitted_pl20.alpha)
fitted_pl40 = powerlaw.Fit(avalprobs40[ii40])
print(fitted_pl40.alpha)

plt.title("Average size pdf")
plt.ylabel("P(s)")
plt.xlabel("s")

plt.legend(['normal', '80%', '60%'])
plt.show()

## Barabasi network

In [ ]:
# Create a Barabasi network of 50 nodes and start simulation
ba_adj_mat, ba_neg_nodes = create_network(size=50, neighbours=5, frac_neg=0.5, net_type='barabasi')
ba_all_act, v, u, I = izhikevich(ba_adj_mat, ba_neg_nodes, time=100000)

# Delete 2 percentages of nodes from network
ba_am20, ba_nn20 = del_rand(ba_adj_mat, ba_neg_nodes, 20, 'node')
ba_am40, ba_nn40 = del_rand(ba_adj_mat, ba_neg_nodes, 40, 'node')

# Run the simulation for the new network
ba_aa20, _ , _, _ = izhikevich(ba_am20, ba_nn20, time=100000)
ba_aa40, _ , _, _ = izhikevich(ba_am40, ba_nn40, time=100000)

#Plot probability of avalance size and calculate power law fit
fig = plt.figure()

ba_aval_times, ba_aval_sizes = get_avalanches(ba_all_act)
ba_y = np.bincount(np.array(ba_aval_sizes))
ba_ii = np.nonzero(ba_y)[0]
ba_avalprobs = ba_y/len(ba_aval_sizes)

ba_aval_times20, ba_aval_sizes20 = get_avalanches(ba_aa20)
ba_y20 = np.bincount(np.array(ba_aval_sizes20))
ba_ii20 = np.nonzero(ba_y20)[0]
ba_avalprobs20 = ba_y20/len(ba_aval_sizes20)

ba_aval_times40, ba_aval_sizes40 = get_avalanches(ba_aa40)
ba_y40 = np.bincount(np.array(ba_aval_sizes40))
ba_ii40 = np.nonzero(ba_y40)[0]
ba_avalprobs40 = ba_y40/len(ba_aval_sizes40)

plt.loglog(ba_ii, ba_avalprobs[ba_ii], '*', ba_ii20, ba_avalprobs20[ba_ii20], '+', ba_ii40, ba_avalprobs40[ba_ii40], 'o')

ba_fitted_pl = powerlaw.Fit(ba_avalprobs[ba_ii])
print(ba_fitted_pl.alpha)
ba_fitted_pl20 = powerlaw.Fit(ba_avalprobs20[ba_ii20])
print(ba_fitted_pl20.alpha)
ba_fitted_pl40 = powerlaw.Fit(ba_avalprobs40[ba_ii40])
print(ba_fitted_pl40.alpha)

plt.title("Average size pdf")
plt.ylabel("P(s)")
plt.xlabel("s")

plt.legend(['normal', '80%', '60%'])
plt.show()

## Ageing network

In [ ]:
# Create a Ageing network of 50 nodes and start simulation
ag_adj_mat, ag_neg_nodes = create_network(size=50, neighbours=5, frac_neg=0.5, net_type='aging')
ag_all_act, v, u, I = izhikevich(ag_adj_mat, ag_neg_nodes, time=100000)

# Delete 2 percentages of nodes from network
ag_am20, ag_nn20 = del_rand(ag_adj_mat, ag_neg_nodes, 20, 'node')
ag_am40, ag_nn40 = del_rand(ag_adj_mat, ag_neg_nodes, 40, 'node')

# Run the simulation for the new network
ag_aa20, _ , _, _ = izhikevich(ag_am20, ag_nn20, time=100000)
ag_aa40, _ , _, _ = izhikevich(ag_am40, ag_nn40, time=100000)

#Plot probability of avalance size and calculate power law fit
fig = plt.figure()

ag_aval_times, ag_aval_sizes = get_avalanches(ag_all_act)
ag_y = np.bincount(np.array(ag_aval_sizes))
ag_ii = np.nonzero(ag_y)[0]
ag_avalprobs = ag_y/len(ag_aval_sizes)

ag_aval_times20, ag_aval_sizes20 = get_avalanches(ag_aa20)
ag_y20 = np.bincount(np.array(ag_aval_sizes20))
ag_ii20 = np.nonzero(ag_y20)[0]
ag_avalprobs20 = ag_y20/len(ag_aval_sizes20)

ag_aval_times40, ag_aval_sizes40 = get_avalanches(ag_aa40)
ag_y40 = np.bincount(np.array(ag_aval_sizes40))
ag_ii40 = np.nonzero(ag_y40)[0]
ag_avalprobs40 = ag_y40/len(ag_aval_sizes40)

plt.loglog(ag_ii, ag_avalprobs[ag_ii], '*', ag_ii20, ag_avalprobs20[ag_ii20], '+', ag_ii40, ag_avalprobs40[ag_ii40], 'o')

ag_fitted_pl = powerlaw.Fit(ag_avalprobs[ag_ii])
print(ag_fitted_pl.alpha)
ag_fitted_pl20 = powerlaw.Fit(ag_avalprobs20[ag_ii20])
print(ag_fitted_pl20.alpha)
ag_fitted_pl40 = powerlaw.Fit(ag_avalprobs40[ag_ii40])
print(ag_fitted_pl40.alpha)

plt.title("Average size pdf")
plt.ylabel("P(s)")
plt.xlabel("s")

plt.legend(['normal', '80%', '60%'])
plt.show()